In [1]:
from agential.cog.reflexion.strategies.qa import ReflexionReActQAStrategy
from agential.llm.llm import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))



In [ ]:
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_INSTRUCTION_HOTPOTQA, REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA


question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"
strat = ReflexionReActQAStrategy(llm=llm)
out = strat.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=1,
    reset=True,
)

In [ ]:
out

In [5]:
from agential.cog.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_COT, GSM8K_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import GSM8K_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT, GSM8K_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_COT_INSTRUCTION_GSM8K, REFLEXION_COT_REFLECT_INSTRUCTION_GSM8K, REFLEXION_REACT_INSTRUCTION_GSM8K, REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K
from agential.cog.reflexion.strategies.math import ReflexionReActMathStrategy

question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = -9867630

strategy = ReflexionReActMathStrategy(llm=llm, testing=True)
out = strategy.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_GSM8K,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=3,
    reset=True,
)

<PROMPT AGENT=============================================>
Answer a math question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be two types:
(1) Calculate[code], which implements code to answer the math question, saving the answer as the `answer` variable.
(2) Finish[code], which returns the code to answer the math question and finishes the task, saving the answer as the `answer` variable.
You have a maximum of 6 steps.

Here are some examples:
Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Thought 1: First, I need to find out how many lollipops Jason gave to Denny.
Action 1: Calculate[
```python
jason_lollipops_initial = 20
jason_lollipops_after = 12
answer = jason_lollipops_after - jason_lollipops_initial
```
]
Observation 1: 
```python
jason_lollipops_initial = 20
jason_lollipops_after = 12
answe

In [6]:
from agential.llm.llm import MockLLM


responses = [
    "First, I need to calculate the total number of eggs laid per day after Janet eats three for breakfast.\nAction 1: Calculate[\n```python\neggs_laid_per_day = 16\neggs_eaten_for_breakfast = 3\neggs_remaining = eggs_laid_per_day - eggs_eaten_for_breakfast\n```\n]\nObservation 1:\n```python\neggs_laid_per_day = 16\neggs_eaten_for_breakfast = 3\neggs_remaining = eggs_laid_per_day - eggs_eaten_for_breakfast\n```\nExecution Status: Done\nOutput: eggs_remaining = 13\nThought 2: I have determined the number of eggs remaining after breakfast. Now, I need to calculate how much Janet earns daily at the farmers' market with the remaining eggs.\nAction 2: Calculate[\n```python\neggs_remaining = 13\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n```\n]\nObservation 2:\n```python\neggs_remaining = 13\nprice_per_egg = 2\nearnings_per_day = eggs_remaining * price_per_egg\n```\nExecution Status: Done\nOutput: earnings_per_day = 26\nThought 3: Janet makes $26 every day at the farmers' market.\nAction 3: Finish[\n```python\nanswer = 26\n```\n]\nObservation 3:\n```python\nanswer = 26\n```",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed = 3\neggs_remaining = eggs_laid_per_day - eggs_consumed\n```\n]',
    "I need to calculate how much money Janet makes from selling the remaining eggs at the farmers' market daily.\nAction 2: Calculate[\n```python\neggs_remaining = 13\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\n```\n]\nObservation 2: \n```python\neggs_remaining = 13\nprice_per_egg = 2\nmoney_made_daily = eggs_remaining * price_per_egg\n```\nExecution Status: Done\nOutput: answer = 26\nThought 3: Janet makes $26 every day at the farmers' market.\nAction 3: Finish[\n```python\nanswer = 26\n```\n]\nObservation 3: \n```python\nanswer = 26\n```",
    "Calculate[\n```python\neggs_remaining = 13  # from previous calculation\negg_price = 2\nmoney_per_day = eggs_remaining * egg_price\nanswer = money_per_day\n```\n]\nObservation 2: \n```python\neggs_remaining = 13\negg_price = 2\nmoney_per_day = eggs_remaining * egg_price\nanswer = money_per_day\n```\nExecution Status: Done\nOutput: answer = 26\nThought 3: Janet makes $26 every day by selling the remaining duck eggs at the farmers' market.\nAction 3: Finish[\n```python\nanswer = 26\n```\n]\nObservation 3: \n```python\nanswer = 26\n```",
    "Janet makes $26 every day at the farmers' market.\nAction 3: Finish[\n```python\nanswer = 26\n```\n]\nObservation 3: \n```python\nanswer = 26\n```",
    'Finish[\n```python\nanswer = 26\n```\n]\nObservation 3: \n```python\nanswer = 26\n```',
    "My reasoning failed because I overlooked the step where Janet bakes muffins for her friends daily before selling the remainder at the farmers' market. In the future, I should ensure to include this intermediate step in my calculations for accurate results. To mitigate this failure, I will revise my plan by accounting for the number of eggs used for baking muffins before calculating the money made at the farmers' market.",
    "First, I need to calculate how many eggs are left after Janet eats breakfast and bakes muffins for her friends.\nAction 1: Calculate[\n```python\neggs_laid_per_day = 16\neggs_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_breakfast - eggs_used_for_muffins\n```\n]\nObservation 1: \n```python\neggs_laid_per_day = 16\neggs_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_breakfast - eggs_used_for_muffins\n```\nExecution Status: Done\nOutput: eggs_remaining = -4933815\nThought 2: My calculation for eggs remaining is incorrect. I mistakenly subtracted the number of eggs used for muffins instead of adding it. I need to modify the calculation to add the eggs used for muffins.\nAction 2: Calculate[\n```python\neggs_laid_per_day = 16\neggs_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_breakfast + eggs_used_for_muffins\n```\n]\nObservation 2: \n```python\neggs_laid_per_day = 16\neggs_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_breakfast + eggs_used_for_muffins\n```\nExecution Status: Done\nOutput: eggs_remaining = 4933821\nThought 3: Now that I have the correct number of eggs remaining, I can calculate how much money Janet makes at the farmers' market daily.\nAction 3: Finish[\n```python\nmoney_per_egg = 2\nmoney_made_per_day = eggs_remaining * money_per_egg\nanswer = money_made_per_day\n```\n]\nObservation 3: \n```python\nanswer = 9867642\n```",
    "Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n```\n]\nObservation 1: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n```\nExecution Status: Done\nOutput: eggs_remaining = -4933815\nThought 2: The calculation result is not logically correct as the number of eggs remaining cannot be negative. I need to revise the plan to accurately compute the eggs remaining.\nAction 2: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_bought_at_market = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_sold_at_market = eggs_bought_at_market - eggs_used_for_muffins\ndollars_per_egg = 2\nmoney_made_per_day = eggs_sold_at_market * dollars_per_egg\n```\n]\nObservation 2: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_bought_at_market = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_sold_at_market = eggs_bought_at_market - eggs_used_for_muffins\ndollars_per_egg = 2\nmoney_made_per_day = eggs_sold_at_market * dollars_per_egg\n```\nExecution Status: Done\nOutput: money_made_per_day = -4933860\nThought 3: The calculated negative amount of money made per day is incorrect. I should review the calculation steps to identify the error and adjust the calculation as needed.\nAction 3: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_sold_at_market = eggs_remaining - eggs_used_for_muffins\ndollars_per_egg = 2\nmoney_made_per_day = max(eggs_sold_at_market, 0) * dollars_per_egg\n```\n]\nObservation 3: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_sold_at_market = eggs_remaining - eggs_used_for_muffins\ndollars_per_egg = 2\nmoney_made_per_day = max(eggs_sold_at_market, 0) * dollars_per_egg\n```\nExecution Status: Done\nOutput: money_made_per_day = 26\nThought 4: The amount of money Janet makes at the farmers' market daily is $26.\nAction 4: Finish[\n```python\nanswer = 26\n```\n]\nObservation 4: \n```python\nanswer = 26\n```",
    "There seems to be an error in the calculation of eggs remaining. I need to adjust my approach to correctly calculate the number of eggs left for Janet to sell at the farmers' market. \nAction 2: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n```\n]\nObservation 2: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n```\nExecution Status: Done\nOutput: answer = -4933815\nThought 3: The calculation result is negative which indicates an error in the logic. I need to revise my plan to accurately determine the number of eggs remaining for Janet to sell at the farmers' market. \nAction 3: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n```\n]\nObservation 3: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n```\nExecution Status: Done\nOutput: answer = -4933815",
    "Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_remaining -= eggs_used_for_muffins\n```\n]\nObservation 2: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_remaining -= eggs_used_for_muffins\n```\nExecution Status: Done\nOutput: answer = -4933807\nThought 3: The answer is negative, indicating an error in the calculation of eggs remaining. I will correct this by ensuring to account for the muffins Janet bakes daily.\nAction 3: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins_per_day = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_remaining -= eggs_used_for_muffins_per_day\n```\n]\nObservation 3: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins_per_day = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_remaining -= eggs_used_for_muffins_per_day\n```\nExecution Status: Done\nOutput: answer = -4933827\nThought 4: The calculation is still not correct. I need to revise my approach to correctly calculate the number of eggs left for selling at the farmers' market.\nAction 4: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\ntotal_eggs_used = eggs_consumed_for_breakfast + eggs_used_for_muffins\neggs_remaining = eggs_laid_per_day - total_eggs_used\n```\n]\nObservation 4: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\ntotal_eggs_used = eggs_consumed_for_breakfast + eggs_used_for_muffins\neggs_remaining = eggs_laid_per_day - total_eggs_used\n```\nExecution Status: Done\nOutput: answer = 10\nThought 5: Janet makes $2 for every fresh duck egg she sells. I need to calculate the total amount she makes daily at the farmers' market.\nAction 5: Calculate[\n```python\nprice_per_egg = 2\ndaily_earnings = price_per_egg * eggs_remaining\n```\n]\nObservation 5: \n```python\nprice_per_egg = 2\ndaily_earnings = price_per_egg * eggs_remaining\n```\nExecution Status: Done\nOutput: answer = 20\nThought 6: Janet makes $20 every day at the farmers' market. This calculation is now correct.\nAction 6: Finish[\n```python\nanswer = 20\n```\n]\nObservation 6: \n```python\nanswer = 20\n```",
    "I have successfully calculated the number of eggs Janet has left for selling. Now, I need to determine how much money she makes from selling these eggs at the farmers' market.\nAction 3: Calculate[\n```python\neggs_remaining = 13  # Updated value after adjustments\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\n```\n]\nObservation 3: \n```python\neggs_remaining = 13  # Updated value after adjustments\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\n```\nExecution Status: Done\nOutput: answer = 26\nThought 4: Janet makes $26 every day at the farmers' market.\nAction 4: Finish[\n```python\nanswer = 26\n```\n]\nObservation 4: \n```python\nanswer = 26\n```",
    'Calculate[\n```python\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\n]\nObservation 3: \n```python\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\nExecution Status: Done\nOutput: answer = -9867647',
    "I need to ensure that the variable `eggs_remaining` is defined before using it to calculate the money made at the farmers' market. I will revise my plan to include this step explicitly. Let's try again.\n\nAction 4: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_remaining -= eggs_used_for_muffins\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\n]\nObservation 4: \n```python\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\nExecution Status: Done\nOutput: answer = -9867646",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast\neggs_remaining -= eggs_used_for_muffins\n\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\n]',
    "The calculation outcome is negative, which doesn't make sense as Janet is selling eggs at the market. I need to review my plan and make sure I'm correctly subtracting the consumed eggs and eggs used for muffins before calculating the money made at the farmers' market. Let's refine the approach.\nAction 5: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\n]\nObservation 5: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\nExecution Status: Done\nOutput: answer = -9867650\nThought 6: There seems to be a recurring error in the calculation process. I will review and ensure that all the deductions are correctly made before calculating the final answer. I will reevaluate the plan to ensure accurate results. Let's correct the calculation approach once more.\nAction 6: Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\n]\nObservation 6: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\n\nprice_per_egg = 2\nmoney_made_per_day = eggs_remaining * price_per_egg\nanswer = money_made_per_day\n```\n]',
    "The outcome is still incorrect. I need to reevaluate my approach and ensure I am correctly calculating the remaining eggs for sale and the money made at the farmers' market. Let's revise the plan to accurately address the issue.",
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\nif eggs_remaining >= 0:\n    price_per_egg = 2\n    money_made_per_day = eggs_remaining * price_per_egg\nelse:\n    money_made_per_day = 0\nanswer = money_made_per_day\n```\n]\nObservation 6: \n```python\neggs_laid_per_day = 16\neggs_consumed_for_breakfast = 3\neggs_used_for_muffins = 4933828\neggs_remaining = eggs_laid_per_day - eggs_consumed_for_breakfast - eggs_used_for_muffins\nif eggs_remaining >= 0:\n    price_per_egg = 2\n    money_made_per_day = eggs_remaining * price_per_egg\nelse:\n    money_made_per_day = 0\nanswer = money_made_per_day\n```',
    'My reasoning failed because I did not properly consider the negative outcome when calculating the remaining eggs. Janet cannot have negative eggs to sell. To mitigate this failure, I should explicitly check for negative values in the remaining eggs calculation and set the money made per day to zero in such cases. This ensures a more accurate representation of the scenario and prevents illogical results.',
    'I need to calculate how many eggs Janet has remaining after taking out breakfast eggs and the ones used for baking muffins before selling the rest at the market.\nAction 1: Calculate[\n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 4933828\nremaining_eggs = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\nif remaining_eggs < 0:\n    remaining_eggs = 0\nmoney_per_day = remaining_eggs * 2\nanswer = money_per_day\n```\n]\nObservation 1: \n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 4933828\nremaining_eggs = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\nif remaining_eggs < 0:\n    remaining_eggs = 0\nmoney_per_day = remaining_eggs * 2\nanswer = money_per_day\n```\nExecution Status: Done\nOutput: answer = 26',
    'Calculate[\n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 4933828\nremaining_eggs = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\nmoney_made_per_day = 2 * remaining_eggs if remaining_eggs >= 0 else 0\nanswer = money_made_per_day\n```\n]',
    "I correctly calculated that Janet makes zero dollars per day at the farmers' market after accounting for breakfast eggs and muffin baking eggs. \nAction 2: Finish[\n```python\nanswer = 0\n```\n]\nObservation 2: \n```python\nanswer = 0\n```",
    'Finish[\n```python\nanswer = 0\n```\n]\nObservation 2: \n```python\nanswer = 0\n```',
]
llm = MockLLM("gpt-3.5-turbo", responses=responses)
strategy = ReflexionReActMathStrategy(llm=llm, testing=True)
out = strategy.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_GSM8K,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=3,
    reset=True,
)

<PROMPT AGENT=============================================>
Answer a math question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be two types:
(1) Calculate[code], which implements code to answer the math question, saving the answer as the `answer` variable.
(2) Finish[code], which returns the code to answer the math question and finishes the task, saving the answer as the `answer` variable.
You have a maximum of 6 steps.

Here are some examples:
Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Thought 1: First, I need to find out how many lollipops Jason gave to Denny.
Action 1: Calculate[
```python
jason_lollipops_initial = 20
jason_lollipops_after = 12
answer = jason_lollipops_after - jason_lollipops_initial
```
]
Observation 1: 
```python
jason_lollipops_initial = 20
jason_lollipops_after = 12
answe

In [7]:
out

ReflexionReActOutput(answer='answer = 0', total_prompt_tokens=240, total_completion_tokens=480, total_tokens=720, total_prompt_cost=0.0003600000000000001, total_completion_cost=0.0009599999999999999, total_cost=0.00132, total_prompt_time=12.0, total_time=0.5, additional_info=[ReflexionReActStepOutput(steps=[ReflexionReActReActStepOutput(thought='First, I need to calculate the total number of eggs laid per day after Janet eats three for breakfast.', action_type='Calculate', query='eggs_laid_per_day = 16\neggs_consumed = 3\neggs_remaining = eggs_laid_per_day - eggs_consumed', observation='\n```python\neggs_laid_per_day = 16\neggs_consumed = 3\neggs_remaining = eggs_laid_per_day - eggs_consumed\n```\nExecution Status: Done\nOutput: answer = None', answer='eggs_laid_per_day = 16\neggs_consumed = 3\neggs_remaining = eggs_laid_per_day - eggs_consumed', external_tool_info={'execution_status': 'Done', 'code_answer': None}, is_correct=False, thought_metrics=PromptMetrics(prompt_tokens=10, compl

In [2]:
from agential.cog.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_REACT
from agential.cog.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_REACT
from agential.cog.reflexion.prompts import HUMANEVAL_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_REACT_INSTRUCTION_HUMANEVAL, REFLEXION_REACT_INSTRUCTION_MBPP, REFLEXION_REACT_REFLECT_INSTRUCTION_HUMANEVAL, REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP
from agential.cog.reflexion.strategies.code import ReflexionReActCodeStrategy, ReflexionReActMBPPStrategy


inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
key = f"{inst['test']}\ncheck({inst['entry_point']})"

agent = ReflexionReActMBPPStrategy(
    llm=llm, 
)
out = agent.generate(
    question=question,
    key=key,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_MBPP,
    reflect_examples=MBPP_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_MBPP,
    reflect_strategy="reflexion",
    additional_keys={"tests": key},
    reflect_additional_keys={"tests": key},
    patience=3,
    reset=True,
)

<PROMPT AGENT=============================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[code], which implements the function to answer the question.
(2) Test[code], which implements assert statement test cases to test the implemented code.
(3) Finish[answer], which returns the code implementation and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))

Thought 1: I need to write a function that accepts two t

In [3]:
out.total_tokens

28783

In [4]:
agent.llm.total_tokens

28783